# SPY 50/200-Day Moving Averages and Golden Cross Detection

## Objective
Given a dataset of daily closing prices for the SPDR S&P 500 ETF Trust (SPY) over the last 5 years, compute:

- **50-day moving average (MA50):** average of the last 50 trading-day closes (simple moving average).
- **200-day moving average (MA200):** average of the last 200 trading-day closes (simple moving average).
- **Golden Cross indicator:** a binary field equal to **1 only on the exact date** when **MA50 crosses from below to above MA200**; otherwise **0**.

## Method
1. Load the CSV and parse the `Date` column as a datetime.
2. Sort rows by date ascending (required for rolling-window calculations).
3. Compute simple moving averages using rolling windows:
   - `MA50 = rolling_mean(Close, 50)`
   - `MA200 = rolling_mean(Close, 200)`
4. Identify a Golden Cross using a strict crossover condition:
   - Today: `MA50 > MA200`
   - Yesterday: `MA50 < MA200`
   If both are true, mark `GoldenCross = 1` for today, else 0.

## Output Table
The produced table contains:

| Column | Meaning |
|---|---|
| `Date` | Trading date |
| `Close` | Closing price on that date |
| `MA50` | 50-day simple moving average of `Close` |
| `MA200` | 200-day simple moving average of `Close` |
| `GoldenCross` | 1 on crossover date only, else 0 |

Notes:
- `MA50` will be `NaN` for the first 49 rows.
- `MA200` will be `NaN` for the first 199 rows.
- Golden Cross can only occur on rows where both MAs exist.

In [1]:
import pandas as pd
import numpy as np

# Load
df = pd.read_csv("SPY_close_price_5Y.csv")

# Standardize columns (assumes Date + Close exist; adjust if your file differs)
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Moving averages (simple)
df["MA50"] = df["Close"].rolling(window=50).mean()
df["MA200"] = df["Close"].rolling(window=200).mean()

# Golden Cross: MA50 crosses from below to above MA200 (strict cross)
cross_up = (df["MA50"] > df["MA200"]) & (df["MA50"].shift(1) < df["MA200"].shift(1))
df["GoldenCross"] = cross_up.astype(int)

# Final table
out = df[["Date", "Close", "MA50", "MA200", "GoldenCross"]].copy()

# Save
out.to_csv("SPY_moving_averages_golden_cross.csv", index=False)

# Optional: print Golden Cross dates
events = out[out["GoldenCross"] == 1][["Date", "Close", "MA50", "MA200"]]
print(events.to_string(index=False))


      Date  Close     MA50     MA200
2023-02-02 416.78 394.3504 394.25025
2025-07-01 617.65 583.0978 582.04155


In [2]:
import pandas as pd

df = pd.read_csv("SPY_close_price_5Y.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

df["MA50"] = df["Close"].rolling(50).mean()
df["MA200"] = df["Close"].rolling(200).mean()

golden_cross = (df["MA50"] > df["MA200"]) & (df["MA50"].shift(1) < df["MA200"].shift(1))
most_recent_close = df.loc[golden_cross, "Close"].iloc[-1]

print(float(most_recent_close))


617.65
